In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent
import os,sys
from sklearn.metrics import precision_recall_curve,confusion_matrix,precision_score,recall_score

In [2]:
def get_files_scan_one_cpu(dir):
    """Get the path of all the files in nested folders"""
    all_files = []
    for entry in os.scandir(dir):
        if entry.is_file():
            all_files.append(entry.path)
        elif entry.is_dir():
            all_files.extend(get_files_scan(entry.path))
    if len(all_files) > 0:
        return all_files
    else:
        raise FileNotFoundError("No Files found. Please check your directory.")

def scan_files_in_folder(folder):
    """Scans files in a folder and returns a list of file paths."""
    files = []
    for entry in os.scandir(folder):
        if entry.is_file():
            files.append(entry.path)
        elif entry.is_dir():
            files.extend(scan_files_in_folder(entry.path))
    return files

def get_files_scan(dir,max_workers=1):
    """Get the path of all the files in nested folders using concurrent.futures."""
    all_files = []
    if max_workers==1:
        return get_files_scan_one_cpu(dir)
    else:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_dir = {executor.submit(scan_files_in_folder, entry.path): entry.path for entry in os.scandir(dir)}
            for future in concurrent.futures.as_completed(future_to_dir):
                folder = future_to_dir[future]
                try:
                    all_files.extend(future.result())
                except Exception as e:
                    print(f"An error occurred while scanning {folder}: {e}")

        if len(all_files) > 0:
            return all_files
        else:
            raise FileNotFoundError("No Files found. Please check your directory.")

In [3]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_7"
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_12/"

# CC3D - Boundary (Model Trained on CC3D and Tested on CC3D)

In [4]:
cc3d_bndry_gt_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","gt"),4)
cc3d_bndry_pred_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"),4)

NameError: name 'os' is not defined

In [34]:
all_precision_scores=[]
all_recall_scores=[]
for i in range(len(cc3d_bndry_gt_files)):
    gt_bndry=np.load(cc3d_bndry_gt_files[i])
    pred_bndry=np.load("/".join(cc3d_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_bndry_gt_files[i].split("/")[-3:]))
    
    all_precision_scores.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores.append(recall_score(gt_bndry,pred_bndry))

In [56]:
print(f"CC3D Boundary Detection Precision: {np.mean(all_precision_scores)}")
print(f"CC3D Boundary Detection Recall: {np.mean(all_recall_scores)}")

CC3D Boundary Detection Precision: 0.7342675813925931
CC3D Boundary Detection Recall: 0.679637856415947


# CC3D - JNC (Model Trained on CC3D and Tested on CC3D)

In [41]:
cc3d_jnc_gt_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"jnc","gt"),4)
cc3d_jnc_pred_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"jnc","pred"),4)

In [42]:
all_precision_scores_jnc=[]
all_recall_scores_jnc=[]
for i in range(len(cc3d_jnc_gt_files)):
    gt_jnc=np.load(cc3d_jnc_gt_files[i])
    pred_jnc=np.load("/".join(cc3d_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_jnc_gt_files[i].split("/")[-3:]))
    
    all_precision_scores_jnc.append(precision_score(gt_jnc,pred_jnc))
    all_recall_scores_jnc.append(recall_score(gt_jnc,pred_jnc))

In [57]:
print(f"CC3D Junction Detection Precision: {np.mean(all_precision_scores_jnc)}")
print(f"CC3D Junction Detection Recall: {np.mean(all_recall_scores_jnc)}")

CC3D Junction Detection Precision: 0.15777425107907497
CC3D Junction Detection Recall: 0.7959275712926798


# ABC - BNDRY (Model Trained on ABC and Tested on ABC)

In [4]:
abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","pred"),4)

In [82]:
all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
for i in range(len(abc_bndry_gt_files)):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [61]:
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry)}")
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry)}")

ABC Boundary Detection Precision: 0.646874226634183
ABC Boundary Detection Recall: 0.5292544282680605


# ABC - JNC (Model Trained on ABC and Tested on ABC)

In [59]:
abc_jnc_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH,"jnc","gt"),4)
abc_jnc_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH,"jnc","pred"),4)

In [60]:
all_precision_scores_abc_jnc=[]
all_recall_scores_abc_jnc=[]
for i in range(len(abc_jnc_gt_files)):
    gt_jnc=np.load(abc_jnc_gt_files[i])
    pred_jnc=np.load("/".join(abc_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_jnc_gt_files[i].split("/")[-3:]))
    
    all_precision_scores_abc_jnc.append(precision_score(gt_jnc,pred_jnc))
    all_recall_scores_abc_jnc.append(recall_score(gt_jnc,pred_jnc))

In [62]:
print(f"ABC Junction Detection Precision: {np.mean(all_precision_scores_abc_jnc)}")
print(f"ABC Junction Detection Recall: {np.mean(all_recall_scores_abc_jnc)}")

ABC Junction Detection Precision: 0.2275767590777098
ABC Junction Detection Recall: 0.7578537525156863


# Summary

In [79]:
print("CC3D (Model Trained on CC3D)")
print("   Boundary-->")
print(f"     Precision: {np.mean(all_precision_scores).round(4)}")
print(f"     Recall: {np.mean(all_recall_scores).round(4)}")
print("   Junction-->")
print(f"     Precision: {np.mean(all_precision_scores_abc_jnc).round(4)}")
print(f"     Recall: {np.mean(all_recall_scores_abc_jnc).round(4)}")
print("\n")
print("ABC (Model Trained on ABC)")
print("   Boundary-->")
print(f"     Precision: {np.mean(all_precision_scores_abc_bndry).round(4)}")
print(f"     Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print("   Junction-->")
print(f"     Precision: {np.mean(all_precision_scores_jnc).round(4)}")
print(f"     Recall: {np.mean(all_recall_scores_jnc).round(4)}")

CC3D (Model Trained on CC3D)
   Boundary-->
     Precision: 0.7343
     Recall: 0.6796
   Junction-->
     Precision: 0.2276
     Recall: 0.7579


ABC (Model Trained on ABC)
   Boundary-->
     Precision: 0.6469
     Recall: 0.5293
   Junction-->
     Precision: 0.1578
     Recall: 0.7959
